In [ ]:
import qiskit
qiskit.__qiskit_version__

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister

from circuit_generator import get_circuit

In [ ]:
import struct

In [ ]:
# Parameterize this execution: circuitID
circuitID = 103

######## 3x3 trivial circuit
if circuitID == 1:
    n= 3
    qc, num_qubits, layers, num_layers = get_circuit (circuitID, n)


######## 4x4 highly branching circuit
elif circuitID == 2:
    n= 4
    qc, num_qubits, layers, num_layers = get_circuit (circuitID, n)

######## n x n Hadamards circuit
elif circuitID == 3:
    n= 3
    qc, num_qubits, layers, num_layers = get_circuit (circuitID, n)

####### circuits 4 and 5: different numbers of Hadamard: see 3

######## 4 x 4 Hadamards Ts Hadamards circuit
elif circuitID == 6:
    n= 4
    qc, num_qubits, layers, num_layers = get_circuit (circuitID, n)

######## variational EfficientSU2
elif circuitID == 19:
    n= 3
    reps = 1
    qc, num_qubits, layers, num_layers = get_circuit (circuitID, n, reps)

######## deeper variational EfficientSU2
elif circuitID == 191:
    n= 3
    reps = 2
    qc, num_qubits, layers, num_layers = get_circuit (19, n, reps)

######## deeper and larger variational EfficientSU2
elif circuitID == 192:
    n= 4
    reps = 2
    qc, num_qubits, layers, num_layers = get_circuit (19, n, reps)

######## deeper and larger variational EfficientSU2
elif circuitID == 193:
    n= 5
    reps = 1
    qc, num_qubits, layers, num_layers = get_circuit (19, n, reps)

######## deeper and larger variational EfficientSU2
elif circuitID == 194:
    n= 6
    reps = 1
    qc, num_qubits, layers, num_layers = get_circuit (19, n, reps)


######## random circuit
elif circuitID == 21: 
    n = 6
    depth = 5

    qc, num_qubits, layers, num_layers = get_circuit (circuitID, n, depth)

######## larger random circuit
elif circuitID == 211: 
    n = 7
    depth = 5

    qc, num_qubits, layers, num_layers = get_circuit (21, n, depth)

######## larger random circuit
elif circuitID == 212: 
    n = 8
    depth = 5

    qc, num_qubits, layers, num_layers = get_circuit (21, n, depth)

######## larger random circuit
elif circuitID == 213: 
    n = 9
    depth = 5

    qc, num_qubits, layers, num_layers = get_circuit (21, n, depth)

######## wider random circuit
elif circuitID == 214: 
    n = 15
    depth = 4

    qc, num_qubits, layers, num_layers = get_circuit (21, n, depth)

########### IQP
elif circuitID == 22:
    n= 4
    true_amplitude = 0.250000+0.000000j
    qc, num_qubits, layers, num_layers = get_circuit (circuitID, n)

########### test
elif circuitID == 100:
    true_amplitude = 0.250000+0.000000j
    qc, num_qubits, layers, num_layers = get_circuit (circuitID)
########### test
elif circuitID == 101:
    true_amplitude = 0.250000+0.000000j
    qc, num_qubits, layers, num_layers = get_circuit (circuitID)
########### test
elif circuitID == 102:
    qc, num_qubits, layers, num_layers = get_circuit (circuitID)
########### test
elif circuitID == 103:
    qc, num_qubits, layers, num_layers = get_circuit (circuitID)

#for layer in layers:
#    print (layer)
#    for gate in layer:
#        print ("\tgate type {0} with {1} qubits: {2}".format(gate[0],len(gate[1]), gate[1]))
#
#    print ()

In [ ]:

# draw the circuit

qc.draw('mpl')

In [ ]:
# file Name
fileName = "circuit_" + str(circuitID) + ".data"
print ("This has {0} qubits and {1} layers".format(num_qubits, num_layers))
print ("Saving circuit {0} to file {1}".format(circuitID,fileName))

In [ ]:
# Saving the circuit

# Open
with open(fileName, 'wb') as outfile:  

    # The  first struct characterizes the circuit height (num_qubits) and width (num_layers)
    # In C struct {int num_qubits, int num_layers}
    outfile.write(struct.pack('=ii', num_qubits, num_layers))
    
    # Now iterate over layers to save the respective gates
    for l,layer in enumerate(layers):
        # Let us first count the number of:
        #. 1 qubit 0 parameters gates   G1P0
        #. 1 qubit 1 parameter gates    G1P1
        #. 2 qubits 0 parameters gates  G2P0
        #. 2 qubits 1 parameters gates  G2P1
        # NOT: ignore repeated gates and 'id' gates
        taken_qubits = []
        GList = [[],[],[],[]]    # G1P0,G1P1,G2P0,G2P1
        for gate in layer:
            gtype = gate[0]

            #if gtype == 'id': continue   # ignore identity

            new_gate = True   # is this a repeated gate ?
                              # Note that this happens with
                              # multiple qubit gates
                              # due to legacy reasons
            for qb in gate[1]:    # qubits this gate acts upon
                if qb in taken_qubits:
                    new_gate = False
                    break
            
            if not new_gate: continue
                
            # remember these qubits heve been used 
            taken_qubits += gate[1]
                
            if len(gate[1])==1 and gate[2]==[]:
                GList[0].append(gate)
            elif len(gate[1])==1 and gate[2]!=[]:
                GList[1].append(gate)
            elif len(gate[1])==2 and gate[2]==[]:
                GList[2].append(gate)
            elif len(gate[1])==2 and gate[2]!=[]:
                GList[3].append(gate)
 
        # For each layer the  1st structure indicates 
        # how many gates of each different count (qubits and params)
        # there are
        # In C struct {int G1P0,G1P1,G2P0,G2P1}
        outfile.write(struct.pack('=iiii', len(GList[0]), len(GList[1]), len(GList[2]), len(GList[3])))
        
        # Now let us save each gate, sorted by type: G1P0,G1P1,G2P0,G2P1
        for idx, gtl in enumerate(GList):   # G1P0,G1P1,G2P0,G2P1
            if (idx==0):       # G1P0
                # these are 1 qubit gates without parameters
                # Gate names coded as integers
                # 'id' - 0
                # 'h'  - 1
                # 'x'  - 2
                # 'y'  - 3
                # 'z'  - 4
                # 's'  - 5
                # 't'  - 6
                # C struct struct {char[] type, int qubit }
                for gate in gtl:   # each G1P0 gate
                    if gate[0]=='id':      # Identity
                        gname = 0 
                    elif gate[0]== 'h':      # Hadamard
                        gname = 1 
                    elif gate[0]== 'x':      # X
                        gname = 2 
                    elif gate[0]== 'y':      # Y
                        gname = 3 
                    elif gate[0]== 'z':      # Z
                        gname = 4 
                    elif gate[0]== 's':      # S
                        gname = 5 
                    elif gate[0]== 't':      # T
                        gname = 6 
                    else:        # Unknown gate
                        print ("Error: Unsupported Gate!!!!")
                        gname = 0 
                    #print ("Layer {0}, gate {1}, qubit {2}".format(l,gname,gate[1][0]))
                    outfile.write(struct.pack('=ii', gname, gate[1][0]))
            elif (idx==1):       # G1P1
                # these are 1 qubit gates with 1 parameter
                # Gate names coded as integers
                # 'rx'  - 11
                # 'ry'  - 12
                # 'rz'  - 13
                # 'p'  - 14
                # C struct struct {char[] type, int qubit, float param, float 8x matrix}
                for gate in gtl:   # each G1P1 gate
                    if gate[0]=='rx':      # RX
                        gname = 11 
                    elif gate[0]== 'ry':      # RY
                        gname = 12 
                    elif gate[0]== 'rz':      # RZ
                        gname = 13 
                    elif gate[0]== 'p':      # P
                        gname = 14 
                    else:        # Unknown gate
                        print ("Error: Unsupported Gate!!!!")
                        gname = 0 
                    #print ("Layer {0}, gate {1}, param {2}, m[0][0]I {3}, pdf[0][0] {4}".format(l,gname,gate[2][0], gate[3][0][0].imag, , gate[4][0][0]))
                    outfile.write(struct.pack('=iifffffffff', gname, gate[1][0], gate[2][0],
                                             gate[3][0][0].real, gate[3][0][0].imag,
                                             gate[3][0][1].real, gate[3][0][1].imag,
                                             gate[3][1][0].real, gate[3][1][0].imag,
                                             gate[3][1][1].real, gate[3][1][1].imag))
            elif (idx==2):       # G2P0
                # these are 2 qubit gates without parameters
                # Gate names coded as integers
                # 'id2'  - 20    Used for errors
                # 'cx'  - 21
                # 'cz'  - 22
                # C struct struct {char[] type, int c_qubit, int t_qubit }
                for gate in gtl:   # each G1P0 gate
                    if gate[0]=='cx':      # Identity
                        gname = 21 
                    elif gate[0]== 'cz':      # Hadamard
                        gname = 22 
                    else:        # Unknown gate
                        print ("Error: Unsupported Gate!!!!")
                        gname = 20 
                    #print ("Layer {0}, gate {1}, c_qubit {2}, t_qubit {3}".format(l,gname,gate[1][0],gate[1][1]))
                    outfile.write(struct.pack('=iii', gname, gate[1][0], gate[1][1]))
            elif (idx==3):       # G2P1
                # these are 2 qubits gates with 1 parameter
                # Gate names coded as integers
                # 'cp'  - 31
                # C struct struct {char[] type, int c-qubit, int t-qubit, float param, float 32x matrix}
                for gate in gtl:   # each G2P1 gate
                    if gate[0]=='cp':      # CP
                        gname = 31 
                    else:        # Unknown gate
                        print ("Error: Unsupported Gate!!!!")
                        gname = 20 
                    #print ("Layer {0}, gate {1}, param {2}, m[0][0]I {3}".format(l,gname,gate[2][0], gate[3][0][0].imag))
                    outfile.write(struct.pack('=iiifffffffffffffffffffffffffffffffff', 
                                              gname, gate[1][0], gate[1][1], 
                                             gate[2][0],
                                             gate[3][0][0].real, gate[3][0][0].imag,
                                             gate[3][0][1].real, gate[3][0][1].imag,
                                             gate[3][0][2].real, gate[3][0][2].imag,
                                             gate[3][0][3].real, gate[3][0][3].imag,
                                             gate[3][1][0].real, gate[3][1][0].imag,
                                             gate[3][1][1].real, gate[3][1][1].imag,
                                             gate[3][1][2].real, gate[3][1][2].imag,
                                             gate[3][1][3].real, gate[3][1][3].imag,
                                             gate[3][2][0].real, gate[3][2][0].imag,
                                             gate[3][2][1].real, gate[3][2][1].imag,
                                             gate[3][2][2].real, gate[3][2][2].imag,
                                             gate[3][2][3].real, gate[3][2][3].imag,
                                             gate[3][3][0].real, gate[3][3][0].imag,
                                             gate[3][3][1].real, gate[3][3][1].imag,
                                             gate[3][3][2].real, gate[3][3][2].imag,
                                             gate[3][3][3].real, gate[3][3][3].imag))
            
    
#  The file gets closed by finishing "with"

In [ ]:
print ("That\'s all, folks!")